In [ ]:
using ImageTransformations
using LinearAlgebra
using Statistics
using Colors
using Random

In [ ]:
function raw2array(path)
    io = open(path, "r")
    raw_data = read(io)
    close(io)
    len = length(raw_data)
    raiz = trunc(Int64, sqrt(len))
    if raiz == 128 # If the image is squared
        return (convert.(Float64, reshape(raw_data, raiz, raiz)))'
    else
        return zeros(128,128)
    end
end

In [ ]:
min_max_scale(a) = (a .- min(a...))/(max(a...)-min(a...)) # The Gray() function only works with 0-1 arrays

In [ ]:
data = readdir("../datasets/rawdata")
number_images = 500
data_slice = shuffle(data)[1:number_images]
tensor = zeros(number_images, 128, 128)
i = 0
for (idx, file) in enumerate(data_slice)
    tensor[idx,:,:] = raw2array("../datasets/rawdata/$(file)")
end

In [ ]:
Gray.(min_max_scale(tensor[1,:,:]))

In [ ]:
# This is the mean face
mean_tensor = mean(tensor, dims = 1)[1,:,:]; # We get a 3D array (1x128x128) but we just need the 128x128
Gray.(min_max_scale(mean_tensor))

In [ ]:
# We have to have 'centered' points in PCA. Let's substract the mean image to all images
tensor_PCA = zeros(number_images, 128, 128)
for idx in 1:number_images
    tensor_PCA[idx,:,:] = tensor[idx,:,:] .- mean_tensor
end
Gray.(min_max_scale(tensor_PCA[1,:,:]))

In [ ]:
function T2M(tensor)
    dims = size(tensor)
    return reshape(tensor, (dims[1], dims[2]*dims[3]))
end

function M2T(matrix)
    dims = size(matrix)
    return reshape(matrix, (dims[1], trunc(Int64, sqrt(dims[2])), trunc(Int64, sqrt(dims[2]))))
end

In [ ]:
X = T2M(tensor_PCA);

In [ ]:
U, Sigma, VT = svd(X)
eigenfaces = M2T(VT'); # The eigenfaces as a tensor

In [ ]:
function reconstruction(original_image, number_eigenf)
    image_as_vector = reshape(original_image, (1,128*128))
    k_coeff = image_as_vector*VT[:, 1:number_eigenf]
    return reshape(k_coeff*VT[:, 1:number_eigenf]', (128,128))
end

In [ ]:
a = reconstruction(tensor[1,:,:], 200)
c = Gray.(min_max_scale(a))